In [1]:
%reload_ext autoreload
from typing import Optional, List, Union
import re
import pandas as pd
import numpy as np
import sys
import os
sys.path.append("..")

from magni_dash.config.constants import TRAJECTORY_SAMPLES_PATH
from magni_dash.visualization.trajectories import load_data

In [2]:
FN = "Qualisys_270922_SC6A_H1_P1_pp.tsv"

In [3]:
df = load_data(os.path.join(TRAJECTORY_SAMPLES_PATH, FN))

2023-02-12 22:59:07.035 
  command:

    streamlit run /home/tmr/miniconda3/envs/magni-dash/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [4]:
df

,Time,DARKO_Robot - 1 X,DARKO_Robot - 1 Y,DARKO_Robot - 2 X,DARKO_Robot - 2 Y,DARKO_Robot - 3 X,DARKO_Robot - 3 Y,DARKO_Robot - 4 X,DARKO_Robot - 4 Y,DARKO_Robot - 5 X,...,DARKO_Robot - 6 X,DARKO_Robot - 6 Y,DARKO_Robot - 7 X,DARKO_Robot - 7 Y,Helmet_1 - 1 X,Helmet_1 - 1 Y,Helmet_1 - 2 X,Helmet_1 - 2 Y,Helmet_1 - 3 X,Helmet_1 - 3 Y
Frame,,,,,,,,,,,,,,,,,,,,,
1,0.00,1396.537,-214.347,853.281,421.981,1695.296,-151.634,792.148,455.794,1578.882,...,1545.846,467.068,892.264,-131.398,6338.134,586.469,6102.620,535.364,6401.660,379.448
2,0.01,1396.539,-214.431,853.263,421.924,1695.309,-151.614,792.150,455.788,1578.931,...,1545.885,467.390,892.231,-131.363,6338.754,586.201,6103.292,533.905,6403.273,379.573
3,0.02,1396.569,-214.387,853.279,421.936,1695.281,-151.636,792.141,455.777,1578.938,...,1545.883,467.410,892.197,-131.322,6339.407,585.921,6104.045,532.485,6404.934,379.663
4,0.03,1396.522,-214.303,853.302,421.966,1695.295,-151.642,792.158,455.793,1578.994,...,1545.871,467.385,892.261,-131.356,6340.158,585.700,6104.754,531.045,6406.563,379.794
5,0.04,1396.523,-214.298,853.270,421.926,1695.292,-151.613,792.154,455.791,1578.918,...,1545.885,467.381,892.193,-131.345,6340.856,585.441,6105.649,529.621,6408.240,379.899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23853,238.52,1396.537,-214.652,853.341,421.903,1695.438,-151.656,792.166,455.601,1579.007,...,1545.846,467.465,892.033,-131.585,4844.397,126.037,4993.440,-32.378,4658.483,-20.204
23854,238.53,1396.535,-214.659,853.343,421.967,1695.428,-151.646,792.171,455.640,1578.940,...,1545.859,467.443,892.065,-131.566,4838.280,119.167,4987.573,-38.966,4652.360,-27.452
23855,238.54,1396.587,-214.661,853.324,421.921,1695.436,-151.676,792.197,455.629,1578.906,...,1545.879,467.469,892.030,-131.576,4832.322,112.183,4981.692,-45.657,4646.233,-34.690


In [5]:
people = df.filter(regex=("Helmet_.*"))

In [6]:
people = people.copy()
people.loc[:, "Time"] = df["Time"]

In [7]:
people

,Helmet_1 - 1 X,Helmet_1 - 1 Y,Helmet_1 - 2 X,Helmet_1 - 2 Y,Helmet_1 - 3 X,Helmet_1 - 3 Y,Time
Frame,,,,,,,
1,6338.134,586.469,6102.620,535.364,6401.660,379.448,0.00
2,6338.754,586.201,6103.292,533.905,6403.273,379.573,0.01
3,6339.407,585.921,6104.045,532.485,6404.934,379.663,0.02
4,6340.158,585.700,6104.754,531.045,6406.563,379.794,0.03
5,6340.856,585.441,6105.649,529.621,6408.240,379.899,0.04
...,...,...,...,...,...,...,...
23853,4844.397,126.037,4993.440,-32.378,4658.483,-20.204,238.52
23854,4838.280,119.167,4987.573,-38.966,4652.360,-27.452,238.53
23855,4832.322,112.183,4981.692,-45.657,4646.233,-34.690,238.54


In [8]:
pat = r'Helmet_(\d+).*' 

In [9]:
helmets_grouped = people.groupby(people.columns.str.extract(pat, expand=False), axis=1)

In [10]:
for gn, hel_df in helmets_grouped:
    print(gn)

1


In [11]:
hel_df.index

Int64Index([    1,     2,     3,     4,     5,     6,     7,     8,     9,
               10,
            ...
            23848, 23849, 23850, 23851, 23852, 23853, 23854, 23855, 23856,
            23857],
           dtype='int64', name='Frame', length=23857)

In [12]:
robot = df.filter(regex=("DARKO_Robot.*"))
robot = robot.copy()
robot.loc[:, "Time"] = df["Time"]

In [13]:
type(helmets_grouped)

pandas.core.groupby.generic.DataFrameGroupBy

In [14]:
class SpatioTemporalFeatures:
    @staticmethod
    def get_delta_columns(input_df: pd.DataFrame):
        initial_columns = set(input_df.columns)
        out_df = input_df.diff().add_suffix("_delta").fillna(0).sort_index(axis=1)

        delta_cols = set(out_df.columns) - initial_columns
        print(f"{delta_cols} created!")
        return out_df
    
    
    @staticmethod
    def get_displacement(input_df: pd.DataFrame, element_name: str):
        out_df = input_df.copy()
        out_df[f"{element_name}_{input_df.name}_displacement"] = np.sqrt(np.square(out_df).sum(axis=1))/1000 # in mm
        return out_df

    @staticmethod
    def get_speed(
        input_df: pd.DataFrame,
        time_col_name: str,
        out_col_name: Optional[str] = "speed",
    ):
        out_df = input_df.copy()

        if time_col_name not in input_df.columns:
            raise ValueError(f"{time_col_name} not in df's columns")


        delta_df = SpatioTemporalFeatures.get_delta_columns(out_df)
        helmet_pat, darko_pat = r'Helmet_(\d+ - \d).*', r'DARKO_Robot.*'
        helmets_disp = delta_df.groupby(
            delta_df.columns.str.extract(helmet_pat, expand=False), axis=1,
        ).apply(SpatioTemporalFeatures.get_displacement, "Helmet")
        helmets_disp.columns = helmets_disp.columns.droplevel(0)
        #pandas 0.18.0 and higher
        helmets_disp = helmets_disp.rename_axis(None, axis=1)
        return helmets_disp

In [15]:
out_df = SpatioTemporalFeatures.get_speed(df, time_col_name="Time")

{'DARKO_Robot - 1 Y_delta', 'Helmet_1 - 2 Y_delta', 'DARKO_Robot - 7 Y_delta', 'Helmet_1 - 1 X_delta', 'DARKO_Robot - 5 X_delta', 'Helmet_1 - 2 X_delta', 'Time_delta', 'DARKO_Robot - 5 Y_delta', 'DARKO_Robot - 4 X_delta', 'DARKO_Robot - 1 X_delta', 'DARKO_Robot - 6 X_delta', 'Helmet_1 - 1 Y_delta', 'DARKO_Robot - 4 Y_delta', 'Helmet_1 - 3 X_delta', 'Helmet_1 - 3 Y_delta', 'DARKO_Robot - 6 Y_delta', 'DARKO_Robot - 2 X_delta', 'DARKO_Robot - 3 Y_delta', 'DARKO_Robot - 7 X_delta', 'DARKO_Robot - 3 X_delta', 'DARKO_Robot - 2 Y_delta'} created!


In [16]:
out_df.columns

Index(['Helmet_1 - 1 X_delta', 'Helmet_1 - 1 Y_delta',
       'Helmet_1 - 1_displacement', 'Helmet_1 - 2 X_delta',
       'Helmet_1 - 2 Y_delta', 'Helmet_1 - 2_displacement',
       'Helmet_1 - 3 X_delta', 'Helmet_1 - 3 Y_delta',
       'Helmet_1 - 3_displacement'],
      dtype='object')

In [17]:
out_df.head(10)

,Helmet_1 - 1 X_delta,Helmet_1 - 1 Y_delta,Helmet_1 - 1_displacement,Helmet_1 - 2 X_delta,Helmet_1 - 2 Y_delta,Helmet_1 - 2_displacement,Helmet_1 - 3 X_delta,Helmet_1 - 3 Y_delta,Helmet_1 - 3_displacement
Frame,,,,,,,,,
1,0.000,0.000,0.000000,0.000,0.000,0.000000,0.000,0.000,0.000000
2,0.620,-0.268,0.000675,0.672,-1.459,0.001606,1.613,0.125,0.001618
3,0.653,-0.280,0.000710,0.753,-1.420,0.001607,1.661,0.090,0.001663
4,0.751,-0.221,0.000783,0.709,-1.440,0.001605,1.629,0.131,0.001634
5,0.698,-0.259,0.000745,0.895,-1.424,0.001682,1.677,0.105,0.001680
6,0.837,-0.231,0.000868,0.777,-1.366,0.001572,1.681,0.125,0.001686
7,0.824,-0.224,0.000854,0.933,-1.333,0.001627,1.741,0.219,0.001755
8,0.842,-0.141,0.000854,0.891,-1.334,0.001604,1.722,0.147,0.001728
9,0.669,-0.204,0.000699,0.800,-1.316,0.001540,1.534,0.155,0.001542
